In [ ]:
# Import necessary packages
import numpy as np
import pickle
import matplotlib.pyplot as plt

from helpers import generate_data, plot_sols, train_test_data, save_to_file

In [ ]:
# Generate continuation data
# generate_data('contparameters.json', 0.1, 1.0, 0.1, 0.5, 0.05)

In [ ]:
# Save to file 
save_to_file(10)

#### Periodic Solutions && NLFR Curve

In [ ]:
plot_sols('FRF1')